## Assignment 2 - Task 1

**Dataset**: Yoochoose Clicks Dataset (yoochoose-clicks.dat)
This dataset is a set of click events collected from a website of an online retailer. Each record in the dataset has four (4) fields:

 - Session ID - the id of the session. In one session there are one or
   many clicks.
 - Timestamp - the time when the click occurred. 
 - Item ID – the unique identifier of item. 
 - Category – the category of the item.

**Objective:** compute the average time that users stay on items in each category.

In [1]:
# Used to get spark to work on Jupyter Notebook
import findspark
findspark.init()

In [2]:
# Initialize spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSCI316-ass2") \
    .config("spark-master", "local") \
    .getOrCreate()
spark

In [3]:
# Import necessary modules from the pyspark library
from pyspark.sql.types import *
from pyspark.sql.functions import col, when, length, lead, avg
from pyspark.sql.window import Window

In [4]:
# Define the schema with correct types for the data
schema = StructType([
    StructField("SESSION_ID", StringType(), True),
    StructField("TIMESTAMP", TimestampType(), True),
    StructField("ITEM_ID", StringType(), True),
    StructField("CATEGORY", StringType(), True)
])

# Indicate timestamp format
timestampFormat = "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"

# Load the data into a Spark DataFrame and print the schema
df_CD = spark.read.csv("yoochoose-clicks.dat", schema=schema, timestampFormat=timestampFormat)
df_CD.printSchema()

root
 |-- SESSION_ID: string (nullable = true)
 |-- TIMESTAMP: timestamp (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- CATEGORY: string (nullable = true)



In [5]:
# Transform DataFrame into 15 item categories: S, 0, 1 to 12, and B (for any 8-10 digits number)
tr_CD = df_CD.withColumn("TR_CATEGORY", when(length(col("CATEGORY")) > 2, "B").otherwise(col("CATEGORY"))).drop(col("CATEGORY"))
tr_CD.show(5)

+----------+--------------------+---------+-----------+
|SESSION_ID|           TIMESTAMP|  ITEM_ID|TR_CATEGORY|
+----------+--------------------+---------+-----------+
|         1|2014-04-07 10:51:...|214536502|          0|
|         1|2014-04-07 10:54:...|214536500|          0|
|         1|2014-04-07 10:54:...|214536506|          0|
|         1|2014-04-07 10:57:...|214577561|          0|
|         2|2014-04-07 13:56:...|214662742|          0|
+----------+--------------------+---------+-----------+
only showing top 5 rows



In [6]:
# Define a window to partition the data into sessions ordered by timestamp
stay_window = Window().partitionBy(col("SESSION_ID")).orderBy(col("TIMESTAMP"))

# Add a new column which contains the timstamp when the use clicks on the next item
# Remove all rows where there is not next item
lead_CD = tr_CD.withColumn("END_TIME", lead(col("TIMESTAMP")).over(stay_window)).where(col("END_TIME").isNotNull())

# Computer the difference between a user clicking on this item and the next item within a session
diff_CD = lead_CD.withColumn("DIFF_TIME", (col("END_TIME").cast("long") - col("TIMESTAMP").cast("long"))).drop(col("TIMESTAMP")).drop(col("END_TIME"))

diff_CD.show(5)

+----------+---------+-----------+---------+
|SESSION_ID|  ITEM_ID|TR_CATEGORY|DIFF_TIME|
+----------+---------+-----------+---------+
|  10000108|214853850|          S|      626|
|  10000172|214853702|          S|       56|
|  10000172|214853767|          S|       73|
|  10000172|214853730|          S|       38|
|  10000172|214853430|          S|       41|
+----------+---------+-----------+---------+
only showing top 5 rows



In [7]:
# Compute the average time that users stay on items in each category
# Also sort result by the average time
diff_CD.groupBy("TR_CATEGORY").agg(avg("DIFF_TIME").alias("AVG_TIME")).sort("AVG_TIME").show(15)

+-----------+------------------+
|TR_CATEGORY|          AVG_TIME|
+-----------+------------------+
|          3|114.66762465216003|
|         11|135.79717136730991|
|          0|144.46260407551037|
|          S|146.07340013335786|
|          8|158.76721158337224|
|         10|163.00827163233603|
|          4| 163.9375423244749|
|          1|  165.261387111114|
|          B|171.32561433195997|
|          9| 173.8529683045447|
|          7|174.97307903473637|
|          2|176.01387986540195|
|          6|195.67335439413253|
|          5|196.24146278988405|
|         12|231.82737799834575|
+-----------+------------------+

